In [1]:
import pandas as pd
import glob

In [2]:
# df=pd.read_csv('master.csv')

In [3]:
# df.head()

In [4]:
# df2=pd.concat([df['Question']+df['Answer']],ignore_index=True)

In [5]:
# df2=" {Question} : "+df['Question']+" {Answer} : "+df['Answer']

In [6]:
# df2.iloc[0]

In [7]:
# df2.shape

In [1]:
import tiktoken

In [2]:
enc=tiktoken.get_encoding('gpt2')

In [3]:
# txt=''
# for i in range(df2.shape[0]):
#     txt=txt+df2.iloc[i]

In [4]:
import numpy as np

In [5]:
# tokens=[]
# tokens.extend(enc.encode(txt))
# tokens_np=np.array(tokens)
# tokens_np_uint16=tokens_np.astype(np.uint16)
# np.save('numpy_arrays/data',tokens_np_uint16)

In [6]:
import torch

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
buf=np.load('numpy_arrays/data.npy')
buf = buf.astype(np.int32)
buf=torch.tensor(buf,dtype=torch.long)

In [9]:
buf.shape[0]

10673294

In [10]:
from transformers import GPT2LMHeadModel
from torch.optim import AdamW

/home/vamsi/Desktop/projects/Finetuning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model_hf=GPT2LMHeadModel.from_pretrained('gpt2')

In [12]:
def getxys(buf,iter,B,T):
    if(B*T*(iter+1)+1>buf.shape[0]):
        pool=buf[B*T*iter:]+buf[:(B*T*(iter+1))-buf.shape[0]+1]    
    else:
        pool=buf[B*T*iter:B*T*(iter+1)+1]
    x=(pool[:-1]).view(B,T)
    y=(pool[1:]).view(B,T)
    return x,y

In [13]:
B=1
T=512
max_iter=1303

In [14]:
# x,y=getxys(buf,0,B,T)

In [15]:
# x=x.to(device)
# y=y.to(device)

In [16]:
# x.shape

In [17]:
# model_hf=model_hf.to(device)

In [18]:
type(model_hf)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [19]:
model_hf=model_hf.to(device)

In [24]:
# model_hf.state_dict()

In [21]:
optimizer=AdamW(params= model_hf.parameters(),lr=0.01)

In [22]:

for iter in range(2):
    model_hf.train()
    optimizer.zero_grad()
    x,y=getxys(buf,iter,B,T)
    x=x.to(device)
    y=y.to(device)
    op=model_hf(input_ids=x,labels=y)
    # op.loss.retain_grad()
    op.loss.backward()
    optimizer.step()
    
    del x,y
    torch.cuda.empty_cache()    

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
